In [7]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [8]:
!ls

sample_data


In [0]:
import json
token = {"username":"yjtrxj","key":"131bbafb775c8139a4bb07dbc9427ac7"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [14]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [16]:
!kaggle datasets list -s blog

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 51, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 910, in dataset_list_cli
    tag_ids, search, user, mine, page)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 880, in dataset_list
    return [Dataset(d) for d in datasets_list_result]
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 66, in __init__
    self.size = File.get_size(self.totalBytes)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 97, in get_size
    return '%.*f%s' % (precision, size, suffixes[suffix_index])
TypeError: float argument required, not NoneType


In [17]:
!kaggle datasets download -d rtatman/blog-authorship-corpus -p /content/

 95% 274M/290M [00:02<00:00, 107MB/s]
100% 290M/290M [00:03<00:00, 97.3MB/s]


In [18]:
!unzip \blog-authorship-corpus.zip

Archive:  blog-authorship-corpus.zip
  inflating: blogtext.csv            


In [0]:
import pandas as pd
blog = pd.read_csv('blogtext.csv')

In [0]:
#blog_df = blog.sample(frac =0.012,replace=False,random_state=5678)

In [0]:
blog_df = blog.iloc[:8000,:]

In [22]:
blog_df.shape

(8000, 7)

In [23]:
blog_df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [24]:
blog_df['text'].unique()

array(['           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         ',
       '           These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail          ',
       "           In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information

In [25]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
id        8000 non-null int64
gender    8000 non-null object
age       8000 non-null int64
topic     8000 non-null object
sign      8000 non-null object
date      8000 non-null object
text      8000 non-null object
dtypes: int64(2), object(5)
memory usage: 437.6+ KB


In [26]:
blog_df['label'] = blog_df.apply(lambda r: [r['gender'], str(r['age']), r['topic'],r['sign']], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
blog_df.drop(columns=['id','gender','age','topic','sign','date'],inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
blog_df.head(2)

,text,label
0,"Info has been found (+/- 100 pages,...","[male, 15, Student, Leo]"
1,These are the team members: Drewe...,"[male, 15, Student, Leo]"


In [29]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
blog_df['text'] = blog_df['text'].map(lambda s : s.lower())
blog_df['text'] = blog_df['text'].map(lambda s:re.sub('[0-9+-/#()!:\']','',s))
blog_df['text'] = blog_df['text'].map(lambda s :s.strip())
sw = stopwords.words('english')
blog_df['text'] = blog_df['text'].apply(lambda s: ' '.join([word for word in s.split() if word not in (sw)]))
blog_df['text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

0       info found pages mb pdf files wait untill team...
1       team members drewes van der laag urllink mail ...
2       het kader van kernfusie op aarde maak je eigen...
3                                         testing testing
4       thanks yahoos toolbar capture urls popupswhich...
                              ...                        
7995    today good excellent time warming becca downlo...
7996    oh goodness oh goodness oh goodness best news ...
7997    well day wasokay average long tiredblaa well e...
7998    sorry title like comment nothing entry anywayi...
7999    ready love youre sensitive pushover active man...
Name: text, Length: 8000, dtype: object

In [0]:
x = blog_df.text
y = blog_df.label

In [0]:
#you can change the test size & a smaller train size will help your system to fit in count vectorizer otherwise system will crash
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [0]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.3)

In [33]:
print("X_train shape is : ",x_train.shape)
print("X_test shape is : ",x_test.shape)
print("X_val shape is : ",x_val.shape)
print("y_train shape is : ",y_train.shape)
print("y_test shape is : ",y_test.shape)
print("y_val shape is : ",y_val.shape)

X_train shape is :  (4480,)
X_test shape is :  (1600,)
X_val shape is :  (1920,)
y_train shape is :  (4480,)
y_test shape is :  (1600,)
y_val shape is :  (1920,)


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv = CountVectorizer(ngram_range=(1,2))

In [0]:
x_train = cv.fit_transform(x_train)

In [0]:
x_test = cv.transform(x_test)

In [0]:
x_val = cv.transform(x_val)

In [39]:
#just to check the
print(x_val)

  (0, 22939)	1
  (0, 94487)	2
  (0, 95572)	1
  (1, 60860)	1
  (1, 132961)	1
  (1, 177871)	1
  (1, 195379)	1
  (1, 195682)	1
  (1, 197845)	1
  (1, 203340)	1
  (1, 226028)	1
  (1, 231364)	1
  (1, 244436)	1
  (1, 244610)	1
  (1, 255791)	1
  (1, 255867)	1
  (1, 278485)	2
  (2, 9807)	3
  (2, 9821)	1
  (2, 11789)	1
  (2, 14326)	1
  (2, 14445)	1
  (2, 19585)	1
  (2, 24251)	2
  (2, 24271)	2
  :	:
  (1919, 256030)	1
  (1919, 257412)	1
  (1919, 258198)	1
  (1919, 259731)	1
  (1919, 262013)	3
  (1919, 262463)	1
  (1919, 264015)	1
  (1919, 264034)	1
  (1919, 266474)	2
  (1919, 269351)	1
  (1919, 270501)	1
  (1919, 272525)	1
  (1919, 273148)	1
  (1919, 273329)	1
  (1919, 275114)	1
  (1919, 281242)	1
  (1919, 281507)	1
  (1919, 281549)	1
  (1919, 288870)	1
  (1919, 288871)	1
  (1919, 289634)	1
  (1919, 296894)	2
  (1919, 300798)	1
  (1919, 301008)	1
  (1919, 301166)	1


In [0]:
label_counts = dict()
for labels in blog_df.label.values:
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

In [41]:
label_counts

{'13': 9,
 '14': 170,
 '15': 390,
 '16': 73,
 '17': 914,
 '23': 144,
 '24': 378,
 '25': 268,
 '26': 112,
 '27': 691,
 '33': 101,
 '34': 540,
 '35': 2311,
 '36': 1703,
 '37': 19,
 '38': 46,
 '39': 79,
 '41': 14,
 '42': 14,
 '44': 3,
 '45': 14,
 '46': 7,
 'Accounting': 2,
 'Aquarius': 351,
 'Aries': 4140,
 'Arts': 31,
 'Automotive': 14,
 'Banking': 16,
 'BusinessServices': 87,
 'Cancer': 238,
 'Capricorn': 88,
 'Communications-Media': 61,
 'Consulting': 18,
 'Education': 121,
 'Engineering': 119,
 'Fashion': 1616,
 'Gemini': 88,
 'Internet': 93,
 'InvestmentBanking': 70,
 'Law': 3,
 'Leo': 208,
 'Libra': 425,
 'Museums-Libraries': 2,
 'Non-Profit': 47,
 'Pisces': 103,
 'Religion': 9,
 'Sagittarius': 755,
 'Science': 33,
 'Scorpio': 854,
 'Sports-Recreation': 77,
 'Student': 615,
 'Taurus': 709,
 'Technology': 2350,
 'Virgo': 41,
 'female': 3061,
 'indUnk': 2616,
 'male': 4939}

In [42]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)
y_val = mlb.transform(y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Law'] will be ignored
  .format(sorted(unknown, key=str)))


In [43]:
y_test.shape

(1600, 56)

In [44]:
y_train.shape

(4480, 56)

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

clf = LogisticRegression(solver='lbfgs',max_iter=10000)
clf = OneVsRestClassifier(clf)
clfm = OneVsRestClassifier(MultinomialNB())
clf0 = GridSearchCV(DecisionTreeClassifier(),cv=5,param_grid={'max_depth':[80,250,500,750,1000]})
clf1 = GridSearchCV(KNeighborsClassifier(),cv=5,param_grid={'n_neighbors':[5,10,15,20,25]})
clf2 = GridSearchCV(RandomForestClassifier(),cv =5,param_grid={'max_depth':[80,250,500,750,1000]})

In [46]:
%%time
clf.fit(x_train,y_train)

CPU times: user 4min 11s, sys: 6min 5s, total: 10min 17s
Wall time: 2min 41s


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=10000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred = clf.predict(x_test)

In [48]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", clf.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred))
print("Hamming_loss:", hamming_loss(y_test, y_pred))
report = classification_report(y_test,y_pred)
print(report)

Model accuracy is: 0.9671875
Accuracy is: 0.376875
Hamming_loss: 0.04161830357142857
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.50      0.03      0.05        36
           2       0.83      0.26      0.39        93
           3       0.00      0.00      0.00        11
           4       0.73      0.34      0.46       181
           5       1.00      0.02      0.05        42
           6       1.00      0.20      0.33        66
           7       1.00      0.02      0.03        57
           8       0.00      0.00      0.00        19
           9       0.61      0.24      0.35       123
          10       0.83      0.29      0.43        17
          11       1.00      0.59      0.74       108
          12       0.73      0.71      0.72       473
          13       0.89      0.53      0.66       328
          14       0.00      0.00      0.00         7
          15       1.00      0.33      0.50       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_test_re = mlb.inverse_transform(y_test)

In [0]:
y_pred_re = mlb.inverse_transform(y_pred)

In [0]:
label_table = pd.DataFrame({'Predicted':y_pred_re,'Actual':y_test_re})

In [52]:
label_table.head(6)

,Predicted,Actual
0,"(35, Aries, Technology, male)","(35, Aries, Technology, male)"
1,"(36, Aries, Fashion, male)","(36, Aries, Fashion, male)"
2,"(female,)","(17, Scorpio, female, indUnk)"
3,"(female,)","(15, Libra, Student, female)"
4,"(17, Libra, Scorpio, female, indUnk)","(17, Scorpio, female, indUnk)"
5,"(27, 33, Aquarius, InvestmentBanking, Taurus, ...","(33, Aquarius, InvestmentBanking, male)"


In [53]:
clfm.fit(x_train,y_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True),
                    n_jobs=None)

In [0]:
y_predm = clfm.predict(x_test)

In [55]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", clfm.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_predm))
print("Hamming_loss:", hamming_loss(y_test, y_predm))
reportm = classification_report(y_test,y_predm)
print(reportm)

Model accuracy is: 0.7926339285714286
Accuracy is: 0.0725
Hamming_loss: 0.05223214285714286
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       1.00      0.06      0.11        36
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        11
           4       1.00      0.03      0.06       181
           5       0.00      0.00      0.00        42
           6       0.50      0.02      0.03        66
           7       0.00      0.00      0.00        57
           8       0.00      0.00      0.00        19
           9       1.00      0.02      0.05       123
          10       0.00      0.00      0.00        17
          11       1.00      0.03      0.05       108
          12       0.98      0.09      0.16       473
          13       0.93      0.20      0.33       328
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
clf0.fit(x_train,y_train)
print("This is the best score :",clf0.best_score_)
print("This are the best parameters:",clf0.best_params_)
print("This is the best estimator:",clf0.best_estimator_)

This is the best score : 0.4794642857142857
This are the best parameters: {'max_depth': 80}
This is the best estimator: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=80, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [57]:
lr0 = clf0.best_estimator_
lr0.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=80, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
y_pred0 = lr0.predict(x_test)

In [59]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr0.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred0))
print("Hamming_loss:", hamming_loss(y_test, y_pred0))
report0 = classification_report(y_test,y_pred0)
print(report0)

Model accuracy is: 0.9401785714285714
Accuracy is: 0.4775
Hamming_loss: 0.05924107142857143
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.15      0.17      0.16        36
           2       0.16      0.10      0.12        93
           3       0.09      0.09      0.09        11
           4       0.35      0.35      0.35       181
           5       0.22      0.19      0.21        42
           6       0.28      0.15      0.20        66
           7       0.16      0.07      0.10        57
           8       0.00      0.00      0.00        19
           9       0.38      0.37      0.38       123
          10       0.75      0.35      0.48        17
          11       0.93      0.76      0.84       108
          12       0.56      0.81      0.66       473
          13       0.64      0.54      0.59       328
          14       0.00      0.00      0.00         7
          15       0.50      0.56      0.53

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred_re0 = mlb.inverse_transform(y_pred0)

In [0]:
label_table0 = pd.DataFrame({'Predicted':y_pred_re0,'Actual':y_test_re})

In [62]:
label_table0.head(6)

,Predicted,Actual
0,"(35, Aries, Technology, male)","(35, Aries, Technology, male)"
1,"(36, Aries, Fashion, male)","(36, Aries, Fashion, male)"
2,"(36, Aries, Fashion, male)","(17, Scorpio, female, indUnk)"
3,"(36, Aries, Fashion, male)","(15, Libra, Student, female)"
4,"(36, Aries, Fashion, male)","(17, Scorpio, female, indUnk)"
5,"(33, Aquarius, InvestmentBanking, male)","(33, Aquarius, InvestmentBanking, male)"


In [63]:
x_train.shape

(4480, 303947)

In [64]:
y_train.shape

(4480, 56)

In [65]:
clf1.fit(x_train,y_train)
print("This is the best score :",clf1.best_score_)
print("This are the best parameters:",clf1.best_params_)
print("This is the best estimator:",clf1.best_estimator_)

This is the best score : 0.07611607142857144
This are the best parameters: {'n_neighbors': 5}
This is the best estimator: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [66]:
lr1 = clf1.best_estimator_
lr1.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
y_pred1 = lr1.predict(x_test)

In [68]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr1.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred1))
print("Hamming_loss:", hamming_loss(y_test, y_pred1))
report1 = classification_report(y_test,y_pred1)
print(report1)

Model accuracy is: 0.17700892857142858
Accuracy is: 0.081875
Hamming_loss: 0.09214285714285714
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00        36
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        11
           4       0.11      0.35      0.16       181
           5       0.50      0.10      0.16        42
           6       1.00      0.02      0.03        66
           7       0.00      0.00      0.00        57
           8       0.00      0.00      0.00        19
           9       0.33      0.02      0.05       123
          10       0.00      0.00      0.00        17
          11       0.00      0.00      0.00       108
          12       0.17      0.13      0.14       473
          13       0.55      0.07      0.13       328
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred_re1 = mlb.inverse_transform(y_pred1)

In [0]:
label_table1 = pd.DataFrame({'Predicted':y_pred_re1,'Actual':y_test_re})

In [71]:
clf2.fit(x_train,y_train)
print("This is the best score :",clf2.best_score_)
print("This are the best parameters:",clf2.best_params_)
print("This is the best estimator:",clf2.best_estimator_)

This is the best score : 0.2571428571428571
This are the best parameters: {'max_depth': 500}
This is the best estimator: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [72]:
lr2 = clf2.best_estimator_
lr2.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred2 = lr2.predict(x_test)

In [74]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr2.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred2))
print("Hamming_loss:", hamming_loss(y_test, y_pred2))
report2 = classification_report(y_test,y_pred2)
print(report2)

Model accuracy is: 0.9859375
Accuracy is: 0.278125
Hamming_loss: 0.05368303571428571
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       1.00      0.03      0.05        36
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        11
           4       1.00      0.02      0.03       181
           5       0.00      0.00      0.00        42
           6       0.00      0.00      0.00        66
           7       0.00      0.00      0.00        57
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00       123
          10       0.00      0.00      0.00        17
          11       0.88      0.06      0.12       108
          12       0.64      0.78      0.70       473
          13       0.95      0.22      0.36       328
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred_re2 = mlb.inverse_transform(y_pred2)

In [0]:
label_table2 = pd.DataFrame({'Predicted':y_pred_re2,'Actual':y_test_re})

In [77]:
label_table2.head()

,Predicted,Actual
0,"(35, Aries, Technology, male)","(35, Aries, Technology, male)"
1,"(36, Aries, Fashion, male)","(36, Aries, Fashion, male)"
2,"(Aries, male)","(17, Scorpio, female, indUnk)"
3,"(male,)","(15, Libra, Student, female)"
4,"(male,)","(17, Scorpio, female, indUnk)"


Conclusion : - The model accuracy or accuracy achieved in training the models is quite good but when these models are put to test they don't perform that well.Among these classifiers the Decision tree gives a better accuracy wwhereas a random forest & Multinomial shows better Micro,Macro,weighted average scores for precision,recall & ofcourse F1-score.Some more techniques can help us understand & get a better accuracy on the test as well.

